In [12]:
import random
from math import floor, exp, sqrt, pi, cos, sin

In [13]:
# Установка библиотеки sklearn
!pip3 install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [14]:
from sklearn.linear_model import LinearRegression

In [15]:
def round_to_2(x):
    """
    Принимает число и возвращает результат его округления
    до 2 знаков после запятой.
    
    Аргументы:
        x: Число.
        
    Возвращаемое значение:
        Результат округления числа до 2 знаков после запятой.
    """
    
    return round(x, 2)

# Grid search

In [16]:
def score_model(model, x_val, y_val):
    """
    Оценивает точность модели по метрике «среднее отклонение от предсказанного значения».
    
    Аргументы:
        model: Модель.
        x_test: Список объектов тестовой выборки.
        y_test: Список значений предсказываемой характеристики для объектов из тестовой выборки.
                Значение на $i$-й позиции в списке соответствует $i$-му объекту тестовой выборки.
        
    Возвращаемое значение:
        Возвращает точность модели.
    """
    
    y_pred = model.predict(x_val)
    
    res = 0.0

    for i in range(len(y_val)):
        res += abs(y_pred[i] - y_val[i])
    
    return res / len(y_val)

In [17]:
def grid_search_solution(model, 
                         model_configs,
                         x_train, y_train, 
                         x_val, y_val):
    """
    Принимает набор конфигураций модели линейной регрессии и находит лучшую из них
    с помощью алгоритма grid search.
    Для оценки точности модели на валидационной выборке используется функция score_model.
    
    Аргументы:
        model: Модель.
        model_configs: Список конфигураций модели линейной регрессии.
                       Каждая конфигурация — список номеров факторов, 
                       которые используются для обучения модели.
        x_train: Список объектов обучающей выборки.
        y_train: Список значений предсказываемой характеристики для объектов из обучающей выборки.
                 Значение на $i$-й позиции в списке соответствует $i$-му объекту обучающей выборки.
        x_val: Список объектов валидационной выборки.
        y_val: Список значений предсказываемой характеристики для объектов из валидационной выборки.
               Значение на $i$-й позиции в списке соответствует $i$-му объекту валидационной выборки.
        
    Возвращаемое значение:
        Возвращает пару значений: лучшая конфигурация, точность модели с данной конфигурацией 
        на валидационной выборке.
    """

    best_config = []
    best_value_score = -1
    
    for config in model_configs:
        x_train_local = [[row[i] for i in config] for row in x_train]
        x_val_local = [[row[i] for i in config] for row in x_val]

        model.fit(x_train_local, y_train)

        score = score_model(model, x_val_local, y_val)

        if (score < best_value_score or best_value_score == -1):
            best_value_score = score
            best_config = config
        
    return best_config, round_to_2(best_value_score)

In [18]:
def process_data(file_path, frac=0.9):
    data_x_y = []
    
    with open(file_path) as f:
        cnt = 0
        
        for line in f:
            if cnt == 0:
                cnt += 1
                continue
                
            l = line.strip().split(',')
            data_x_y.append(([float(x) for x in l[:-1]], float(l[-1])))  
            
            cnt += 1
            
    cut_n = floor(frac * len(data_x_y))
            
    train_x_y = data_x_y[:cut_n]
    val_x_y = data_x_y[cut_n:]
    
    train_x, train_y = [], []
    
    for x, y in train_x_y:
        train_x.append(x)
        train_y.append(y)
        
    val_x, val_y = [], []
    
    for x, y in val_x_y:
        val_x.append(x)
        val_y.append(y)
    
    return train_x, train_y, val_x, val_y

def grid_search_test():
    def split_df_to_xs_and_ys(df):
        xs = []
        ys = []
        
        for _, row in df.iterrows():
            xs.append([row['x1'], row['x2'], row['x3'], row['x4']])
            ys.append(row['y'])
            
        return xs, ys
        
    data_x_train_example_1 = [[1, -33], [2, -21], [3, -34234]]
    data_y_train_example_1 = [1, 2, 3]
    
    data_x_val_example_1 = [[4, -231], [5, -342341], [6, -23]]
    data_y_val_example_1 = [4, 5, 6]
    
    configs_example_1 = [[0], [1]]
    
    res_example_1 = ([0], 0)
    
    assert grid_search_solution(LinearRegression(),
                                configs_example_1,
                                data_x_train_example_1, data_y_train_example_1,
                                data_x_val_example_1, data_y_val_example_1) == res_example_1
    
    data_x_train_example_2, data_y_train_example_2, \
        data_x_val_example_2, data_y_val_example_2 = process_data('grid_search_test_data.csv', frac=0.9)
    
    configs_example_2 = [[0], [0, 1], [0, 1, 2], [2, 3], [1]]
    
    res_example_2 = ([0, 1, 2], 0.26)
    
    assert grid_search_solution(LinearRegression(),
                                configs_example_2,
                                data_x_train_example_2, data_y_train_example_2,
                                data_x_val_example_2, data_y_val_example_2) == res_example_2

    print('Тест прошёл успешно!')

In [19]:
grid_search_test()

Тест прошёл успешно!


# Метод дихотомии

In [20]:
def dichotomous_search_solution(f, a, b, eps):
    """
    Производит поиск минимума заданной функции в заданном интервале с помощью метода дихотомии.
    
    Аргументы:
        f: Функция от одного аргумента, минимум которой необходимо найти.
        a: Левая граница интервала, в котором происходит поиск минимума.
        b: Правая граница интервала, в котором происходит поиск минимума.
        eps: Допустимая погрешность при поиске минимума.
        
    Возвращаемое значение:
        Возвращает координату точки, в которой достигается минимальное значение функции.
        Координата должна быть округлена до 2 знаков после запятой.
    """

    a_local = a
    b_local = b

    while ((b_local - a_local) > eps * 2):
        mean_value = (a_local + b_local) / 2
        
        left_mean_value = mean_value - eps / 2

        right_mean_value = mean_value + eps / 2

        left = f(left_mean_value)
        right = f(right_mean_value)

        if (left > right):
            a_local = left_mean_value
        else:
            b_local = right_mean_value

    return round_to_2((a_local + b_local) / 2)

In [21]:
def dichotomous_search_test():
    f_example_1 = lambda x: x ** 2
    
    a_example_1 = -2
    b_example_1 = 5
    eps_example_1 = 0.01
    
    res_example_1 = 0
    
    min_example_1 = dichotomous_search_solution(f_example_1, a_example_1, b_example_1, eps_example_1)
    
    if min_example_1 == 0.0:
        min_example_1 = abs(min_example_1)
    
    assert min_example_1 == res_example_1
    
    f_example_2 = lambda x: (exp(2 * (x - 2)) + 1) * (x + 1) ** 2
    
    a_example_2 = -100
    b_example_2 = 200
    eps_example_2 = 0.01
    
    res_example_2 = -1
    
    min_example_2 = dichotomous_search_solution(f_example_2, a_example_2, b_example_2, eps_example_2)
    
    if min_example_2 == 0.0:
        min_example_2 = abs(min_example_2)
    
    assert min_example_2 == res_example_2

    print('Тест прошёл успешно!')

In [22]:
dichotomous_search_test()

Тест прошёл успешно!


# Градиентный спуск

In [23]:
def gradient_descent_solution(grad_f, x_0, alpha, eps):
    """
    Производит поиск минимума заданной функции двух аргументов с помощью градиентного спуска.
    
    Аргументы:
        grad_f: Градиент функции двух аргументов, для которой необходимо найти минимум.
                Функция принимает на вход точку (список из двух значений) и возвращает
                вектор градиента в этой точке (тоже список из двух значений).
        x_0: Стартовая точка (список из двух значений), из которой запускается алгоритм градиентного спуска.
        alpha: Коэффициент скорости обучения.
        eps: Минимальное расстояние между текущей точкой градиентного спуска и следующей,
             при котором работа алгоритма останавливается.
        
    Возвращаемое значение:
        Координаты точки (список из двух значений), в которой достигается минимальное значение функции.
        Каждая координата должна быть округлена до 2 знаков после запятой.
    """
    def dist_v(x1, x2):
        return ((x1[0] - x2[0]) ** 2 + (x1[1] - x2[1]) ** 2) ** (1/2)
    
    def subtraction_vector(_x, _vector):
        x1 = _x[0] - _vector[0]
        x2 = _x[1] - _vector[1]

        return [x1, x2]
    
    def multiplication_vector(_x, _vector):
        x1 = _x * _vector[0]
        x2 = _x * _vector[1]

        return [x1, x2]
    
    def round_coordinates(_x):
        x1 = round_to_2(_x[0])
        x2 = round_to_2(_x[1])

        return [x1, x2]


    x_past = x_0

    x_present = subtraction_vector(x_past, multiplication_vector(alpha, grad_f(x_past)))

    while(dist_v(x_present, x_past) > eps):
        x_past = x_present
        x_present = subtraction_vector(x_past, multiplication_vector(alpha, grad_f(x_past)))

    return round_coordinates(x_present)


In [24]:
def gradient_descent_test():
    f_example_1 = lambda x: 2 * x[0] ** 2 + 2 * x[1] ** 2
    grad_f_example_1 = lambda x: [4 * x[0], 4 * x[1]]
    
    x_0_example_1 = [32, -4]
    
    alpha_example_1 = 0.01
    eps_example_1 = 0.001
    
    res_example_1 = [0.02, 0]
    
    assert gradient_descent_solution(grad_f_example_1, x_0_example_1, alpha_example_1, eps_example_1) == res_example_1
    
    f_example_2 = lambda x: (x[0] - 1) ** 4 + 2 * (x[1] + 2) ** 2 - 1
    grad_f_example_2 = lambda x: [4 * (x[0] - 1) ** 3, 4 * (x[1] + 2)]
    
    x_0_example_2 = [-2, 2]
    
    alpha_example_2 = 0.0001
    eps_example_2 = 0.0001
    
    res_example_2 = [0.58, -1.76]
    
    assert gradient_descent_solution(grad_f_example_2, x_0_example_2, alpha_example_2, eps_example_2) == res_example_2

    print('Тест прошёл успешно!')

In [25]:
gradient_descent_test()

Тест прошёл успешно!


# Метод имитации отжига

In [26]:
def g(x):
    return 2 * x ** 2 + cos(pi * x) - 2.9 * sin(2 * pi * x) + cos(3 * pi * x) * sin(pi * x)

In [27]:
def sub_with_p(random_gen, p, new_x, x):
    """
    С заданной вероятностью возвращает точку `new_x`, в остальных случаях
    возвращает точку x.
    
    Аргументы:
        random_gen: Генератор случайных чисел.
        p: Вероятность, с которой нужно вернуть первую точку.
        new_x: Точка, которая возвращается с вероятностью p.
        x: Точка, которая возвращает в остальных случаях.
        
    Возвращаемое значение:
        Точка, которая с вероятностью p будет равна new_x. В противном случае точка будет равна x.
    """
    
    val = random_gen.random()
    
    if val > p:
        return x
    
    return new_x

In [28]:
def simulated_annealing_solution(f, x_0, t_0, lam, eps, random_gen):
    """
    Производит поиск минимума функции с помощью метода имитации отжига.
    
    Аргументы:
        f: Функция, минимум которой необходимо найти.
        x_0: Стартовая точка, из которой начинается процесс поиска минимума.
        t_0: Начальная температура системы.
        lam: Коэффициент охлаждения системы на каждом шаге.
             Охлаждение происходит по правилу T = lam * T.
        eps: Когда температура становится меньше eps, метод имитации отжига останавливает свою работу.
        random_gen: Генератор случайных чисел.
        
    Возвращаемое значение:
        Точка, которую метод считает минимумом функции.
    """
    
    T = t_0
    x_t = x_0
    
    while T > eps:
        # Случайное изменение текущей точки в пределах (-1, 1)
        new_x = x_t + random_gen.uniform(-1, 1)

        # Вычисляем разницу значений функции
        delta_f = f(new_x) - f(x_t)

        if delta_f < 0:
            # Если новое значение функции меньше, принимаем новую точку
            x_t = new_x
        else:
            # Если значение больше, принимаем с вероятностью e^(-delta_f / T)
            prob = exp(-delta_f / T)
            x_t = sub_with_p(random_gen, prob, new_x, x_t)
        
        # Охлаждаем систему
        T *= lam

    return round_to_2(x_t)

In [29]:
def simulated_annealing_test():
    weierstrass_function = lambda x: 10 + x ** 2 - 10 * cos(2 * pi * x)

    f_example_1 = weierstrass_function
    
    x_0_example_1 = 20
    t_0_example_1 = 1000
    lam_example_1 = 0.99
    eps_example_1 = 0.001
    
    random_gen_example_1 = random.Random(0)
    
    res_example_1 = 0.0
    
    found_min_example_1 = \
        simulated_annealing_solution(weierstrass_function,
                                     x_0_example_1,
                                     t_0_example_1,
                                     lam_example_1,
                                     eps_example_1,
                                     random_gen_example_1)
    
    assert round_to_2(weierstrass_function(found_min_example_1)) == res_example_1
    
    print('Тест прошёл успешно!')

In [30]:
simulated_annealing_test()

Тест прошёл успешно!


In [47]:
x_0 = 10
t_0 = 1000
lam = 0.99
eps = 0.001
random_gen = random.Random(0)

found_min = simulated_annealing_solution(g,
                                        x_0,
                                        t_0,
                                        lam,
                                        eps,
                                        random_gen)

print(round_to_2(found_min))

-0.71
